# **Machine Learning: Project Part 2 - Seal Call Discrimination**

---

**Author: Damien Farrell**

---


### **Objective**
The aim of this project is to analyze a recorded dataset to investigate the feasibility of discriminating between different seal calls. The project is structured in steps to build a machine learning model that can potentially detect seal calls from audio recordings. While creating a fully functional detector is the ultimate goal, it may not be the final outcome of this project.

---

#### **Step A: Data Pre-processing**

This step is completed in notebook `A. data_preprocessing.ipynb`. The file `processed_data.pkl` was produced from this notebook.

#### **Step B: Model Training**

This part is completed in notebook `B. model_selection.ipynb`.

#### **Step C: Refine Model**
1. **Parameter Tuning**:


2. **Validation**:


---

### **Step C: Refine Model** 

---

In [22]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, regularizers
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
import keras_tuner as kt

In [23]:
# Load in processed data
df = pd.read_pickle('processed_data.pkl')
df.head()

# Perform one-hot encoding
df = pd.get_dummies(df, columns=['Annotation'], prefix='Annotation')

In [ ]:
X = df[['snippet_spectrogram']].to_numpy()

# "Flatten" the (3742, 1) array to (3742,)
X = X.ravel()
print(X.shape)

# Stack along a new axis to get shape (3742, 42, 84)
X = np.stack(X, axis=0)
print(X.shape)

# Reshape to 2D for MinMaxScaler
X_reshaped = X.reshape(-1, X.shape[-2])  # Shape: (3742 * 42, 84)

# Apply MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_reshaped)

# Reshape back to the original shape
X = X_scaled.reshape(X.shape)

In [25]:
def plot_loss(history, which='loss'):
    plt.plot(history.history[which], label='train')
    try:
        plt.plot(history.history['val_' + which], label='Validation Loss')
    except:
        pass
    plt.xlabel('Epoch')
    plt.ylabel(which)
    plt.legend()
    plt.grid(True)

In [26]:
def plot_accuracy(history):
    plt.plot(history.history['accuracy'], label='Accuracy')
    try:
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    except:
        pass
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

In [27]:
def confusion_matrix_plot():
    # Predict probabilities and convert to class predictions
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Convert y_test one-hot to integer-encoded labels
    y_true = np.argmax(y_test, axis=1)

    # Extract the real class names
    class_names = [col.replace("Annotation_", "") for col in y_test.columns]

    # Print classification report
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))

    # Plot the confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d",
                xticklabels=class_names,
                yticklabels=class_names,
                cmap="viridis", ax=ax)

    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.title("Confusion Matrix")
    plt.show()

In [28]:
X = np.stack(df['snippet_spectrogram'].values)
y = df[[col for col in df.columns if col.startswith("Annotation_")]]  # Select annotation columns

#### **Model 1: CNN**

In [29]:
# Define focal loss
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = -y_true * alpha * K.pow(1 - y_pred, gamma) * K.log(y_pred)
        return K.mean(loss, axis=-1)
    return focal_loss_fixed

In [30]:
# Early stopping
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

In [31]:
# Build model function for Keras Tuner
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(42, 84, 1)))
    
    # Add convolutional layers
    for i in range(hp.Int("conv_layers", 1, 3)):  # Search for 1-3 Conv layers
        model.add(
            layers.Conv2D(
                filters=hp.Int(f"filters_{i}", min_value=16, max_value=128, step=16),
                kernel_size=hp.Choice(f"kernel_size_{i}", values=[3, 5]),
                padding="same",
                activation="relu",
            )
        )
        model.add(layers.MaxPooling2D())

    # Flatten + Dense
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("dense_units", min_value=64, max_value=256, step=32),
            activation="relu",
        )
    )
    # Adjust final layer based on number of classes (or multi-label dimension)
    num_classes = len(np.unique(stratify_labels))
    model.add(layers.Dense(num_classes, activation="softmax"))

    loss_type = hp.Choice("loss_type", ["focal", "categorical_crossentropy"])
    if loss_type == "focal":
        chosen_loss = focal_loss(gamma=2.0, alpha=0.25)  # Custom focal loss
    else:
        chosen_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", values=[1e-3, 1e-4, 1e-5])
        ),
        loss=chosen_loss,
        metrics=[
            "accuracy",
            tf.keras.metrics.Precision(thresholds=0.5, name="precision"),
            tf.keras.metrics.Recall(thresholds=0.5, name="recall"),
            tf.keras.metrics.AUC(multi_label=True, name="auc"),
            tf.keras.metrics.F1Score(threshold=0.5, average="macro", name="f1_score"),
        ],
    )
    return model

In [32]:
#data
X = np.stack(df['snippet_spectrogram'].values)
y = df[[col for col in df.columns if col.startswith("Annotation_")]].values

stratify_labels = np.argmax(y, axis=1)

In [33]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=stratify_labels
)

In [ ]:
#tuner = kt.Hyperband(
#    build_model,
#    objective=kt.Objective("val_accuracy", direction="max"),
#    max_epochs=20,
#    factor=3,
#    directory="tuner_results",
#    project_name="spectrogram_tuning",
#)

tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=20,  # Explicit number of trials
    directory="tuner_results",
    project_name="bayesian_search",
)


In [ ]:
tuner.search(
    X_train, y_train,
    epochs=10,  # or a different number for the tuning phase
    validation_data=(X_val, y_val),
    callbacks=[early_stop_callback]
)

In [ ]:
# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best loss_type:", best_hp.get("loss"))
print("Best learning_rate:", best_hp.get("learning_rate"))
print("Best conv_layers:", best_hp.get("conv_layers"))

In [ ]:
# build and evaluate the "best model" on the validation set
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[early_stop_callback]
)
val_results = best_model.evaluate(X_val, y_val, verbose=0, return_dict=True)
print("Validation Results after Tuning:", val_results)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []
fold_f1_scores = []

for fold_idx, (train_index, val_index) in enumerate(kf.split(X, stratify_labels), 1):
    print(f"\n--- Fold {fold_idx} ---")
    X_train_cv, X_val_cv = X[train_index], X[val_index]
    y_train_cv, y_val_cv = y[train_index], y[val_index]

    # Build a new model with the best hyperparameters from the tuner
    model_cv = tuner.hypermodel.build(best_hp)

    # You could use a different patience if you like
    early_stop_cv = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )

    # Train the best model on this fold
    history = model_cv.fit(
        X_train_cv, y_train_cv,
        epochs=20,
        validation_data=(X_val_cv, y_val_cv),
        callbacks=[early_stop_cv],
        verbose=1,
    )

    # Evaluate on the validation portion of this fold
    results = model_cv.evaluate(X_val_cv, y_val_cv, verbose=0, return_dict=True)
    fold_accuracies.append(results["accuracy"])
    fold_f1_scores.append(results["f1_score"])

    # Clear session to avoid clutter
    K.clear_session()

In [ ]:
print(f"\nAvg accuracy over folds: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")
print(f"Avg F1 over folds:       {np.mean(fold_f1_scores):.4f} ± {np.std(fold_f1_scores):.4f}")

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot the training & validation accuracy
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 9.1) Predict on the validation set
y_pred_probs = best_model.predict(X_val)

# Convert probabilities to predicted class indices
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Convert one-hot true labels to class indices
y_true_labels = np.argmax(y_val, axis=1)

# 9.2) Classification report
print("\nClassification Report (Validation Set):\n")
print(classification_report(y_true_labels, y_pred_labels))

# 9.3) Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=np.unique(y_true_labels),
            yticklabels=np.unique(y_true_labels))
plt.title('Confusion Matrix (Validation Set)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
final_results = best_model.evaluate(X_val, y_val, verbose=0, return_dict=True)
print("\nFinal Validation Metrics:")
for metric_name, metric_value in final_results.items():
    print(f"{metric_name}: {metric_value:.4f}")

---

### **References**

1. [Bird Song Dataset on Kaggle](https://www.kaggle.com/code/sophiagnetneva/cnn-for-sound-classification-bird-calls-90)
1.
1.
1.
1.
1.
1.
1.
1.





---

# END